# Clase 22: Selección de Características, Reducción de Dimensionalidad y Ajuste de Hiperparámetros

**MDS7202: Laboratorio de Programación Científica para Ciencia de Datos**

**Profesor: Pablo Badilla**

## Objetivos de esta clase

- Comprender la importancia de seleccionar características y reducir dimensionalidad.
- Seleccionar Características relevantes.
- Reducción de dimensionalidad.
- Integrar estas técnicas con `Pipeline`
- Buscar la mejor configuración de hiperparámetros con `GridSearch`

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler

df = pd.read_csv("./resources/descriptores_musica.csv")
df = df.astype({"time_signature": str, "key": str, "mode": str})

---

## Problema de Hoy: 🎸🤘 Caracterización Musical 🎼🎵 

    
Los atributos son: 

- `key`: escala de la canción. 0 = C, 1 = C♯/D♭, 2 = D...  [Mas información](https://en.wikipedia.org/wiki/Pitch_class).
- `modo`: 1 si la escala es mayor, 0 si es menor.
- `time_signature`: cuántos pulsos hay en cada compás. (4, 3,...).
- `loudness`: Volumen de la canción (rango -60, 0).


- `acousticness`: Probabilidad de que la canción sea solo acústica. Valores cercanos a 1 indican que la canción es probablemente acústica.
- `danceability`: Describe que tan bailable es la canción. Valores cercanos a 1 indican que la canción es muy bailable.
- `energy`: Mide que tan energética es una canción. Mide cosas como la rapidez, el volumen y el ruido. Valores cercanos a 1 indican que la canción es muy enégrica.
- `instrumentalness`: Probabilidad que la canción contenga voces. 1 es muy probable que contenga voz.
- `liveness`: Probabilidad de que la canción fuese grabada en vivo. 1 es muy probable que la canción haya sido grabada en vivo.
- `speechiness`: Probabilidad de que la canción contenga palabras habladas (ejemplo: podcast : 1). 
- `valence`: Sentimiento de la canción (rango 0, 1). 1 -> felicidad, alegria, euforia. 0 -> Tristeza, enojo, depresión.
- `tempo` : Pulsos por minuto de la canción (BPM). 


La variable a predecir es: 

- `genre`: Género de la canción.


**Pregunta**: A simple vista,

- ¿Hay carácterísticas que podrían estas repetidas? (**Irrelevante**)
- ¿hay características que nos dicen mas o menos lo mismo? (**Redundante**)


### Análisis Exploratorio de Datos

In [ ]:
import pandas as pd
import plotly.express as px

df = pd.read_csv("./resources/descriptores_musica.csv")
df.head(5)

In [ ]:
df.describe()

In [ ]:
def get_ejemplo(idx):
    """
    Obtiene un ejemplo y lo formatea como columna.
    """
    ejemplo = (
        df.loc[
            idx,
            [
                "danceability",
                "energy",
                "speechiness",
                "acousticness",
                "instrumentalness",
                "valence",
                "name",
                "artist",
                "genre",
            ],
        ]
        .to_frame()
        .reset_index()
    )
    ejemplo.columns = ["Descriptor", "Valor"]
    return ejemplo

In [ ]:
# pueden cambiar el índice de alguno de estos ejemplos para
# mostrar otra canción en la visualización
ejemplo1 = get_ejemplo(102)
ejemplo2 = get_ejemplo(385)
ejemplo3 = get_ejemplo(15)
ejemplo4 = get_ejemplo(484)

ejemplos = [ejemplo1, ejemplo2, ejemplo3, ejemplo4]

#### Spider/Radar Chart

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=2,
    cols=2,
    specs=[
        [{"type": "polar"}, {"type": "polar"}],
        [{"type": "polar"}, {"type": "polar"}],
    ],
    subplot_titles=[
        ejemplo1.loc[6, "Valor"],
        ejemplo3.loc[6, "Valor"],
        ejemplo2.loc[6, "Valor"],
        ejemplo4.loc[6, "Valor"],
    ],
)

for i, ejemplo in enumerate(ejemplos):
    fig.add_trace(
        go.Scatterpolar(
            r=ejemplo.loc[0:5, "Valor"],
            theta=ejemplo.loc[0:5, "Descriptor"],
            fill="toself",
            name=f"{ejemplo.loc[6, 'Valor']} - {ejemplo.loc[7, 'Valor']} ({ejemplo.loc[8, 'Valor']})",
        ),
        col=i // 2 + 1,
        row=i % 2 + 1,
    )

fig.update_layout(
    polar=dict(radialaxis=dict(visible=True, range=[0, 1])),
    showlegend=False,
    title="ScatterPolar/Radar/Spider Chart/ Descripción de Ejemplos",
    height=700,
)

fig.show()

#### Histogramas

In [ ]:
px.histogram(df, x="duration_ms")

In [ ]:
px.histogram(df, x="loudness")

In [ ]:
px.histogram(df, x="tempo")

In [ ]:
dt_to_hists = df.loc[
    :,
    [
        "danceability",
        "energy",
        "speechiness",
        "acousticness",
        "instrumentalness",
        "valence",
        "liveness",
        "genre",
    ],
].melt(id_vars=["genre"], var_name="variable", value_name="valor")

px.histogram(
    dt_to_hists, x="valor", color="variable", facet_col="variable", facet_col_wrap=4
).update_layout(showlegend=False)

In [ ]:
from sklearn.preprocessing import MinMaxScaler, Normalizer
from umap import UMAP

projection_pipe = Pipeline(
    [
        (
            "Column Transformer",
            ColumnTransformer(
                [("MinMax", MinMaxScaler(), ["duration_ms", "loudness"])],
                remainder="passthrough",
            ),
        ),
        ("Normalize", Normalizer()),
        ("UMAP", UMAP(random_state=88, n_neighbors=20, min_dist=0.15)),
    ]
)


projections = projection_pipe.fit_transform(
    df.loc[
        :,
        [
            "danceability",
            "energy",
            "speechiness",
            "acousticness",
            "instrumentalness",
            "valence",
            "liveness",
            "duration_ms",
            "loudness",
        ],
    ]
)
df_proj = pd.DataFrame(projections, columns=["x", "y"])


df_fig = df.copy()
df_fig = pd.concat([df_fig, df_proj], axis=1)
df_fig["hover_name"] = df_fig["artist"] + " - " + df_fig["name"]

fig = px.scatter(
    df_fig,
    x="x",
    y="y",
    color="genre",
    hover_name="hover_name",
    hover_data=[
        "danceability",
        "energy",
        "speechiness",
        "acousticness",
        "instrumentalness",
        "valence",
    ],
)
fig.show()

## Motivación 

### Detalle Interesante 1: Correlación entre las variables.


$$corr (X, Y) = \frac{1}{(s_{x} s_{y})} \sum_{i=1}^{m} (x_i - \overline{x})(y_i - \overline{y}) $$


Los valores varian entre -1 y 1

- Positivo: Relación directa: Crece una, crece la otra. Mientras mayor, mas similares son las variables.
- Negativo: Relación inversa: Crece una, decrece la otra.


In [ ]:
correlations_df = df[
    [
        "danceability",
        "energy",
        "loudness",
        "speechiness",
        "acousticness",
        "instrumentalness",
        "liveness",
        "valence",
        "tempo",
    ]
]
correlations = correlations_df.corr()


px.imshow(
    correlations,
    labels=dict(x="", y="", color="Correlation"),
    x=correlations_df.columns,
    y=correlations_df.columns,
    zmin=-1,
    zmax=1,
    color_continuous_scale="Inferno",
)

### Detalle Interesante 2: Correlación entre las variables de Entrada y la variable por Predecir 

In [ ]:
labels = df.loc[:, ["genre"]]
labels

In [ ]:
encoder = OneHotEncoder(sparse=False)
encoded_labels = encoder.fit_transform(labels)

encoded_labels_df = pd.DataFrame(encoded_labels, columns=encoder.get_feature_names_out())
encoded_labels_df

In [ ]:
# juntamos correlations_df con encoded_labels_df

correlations_labels_df = pd.concat([correlations_df, encoded_labels_df], axis=1)

correlations_labels = correlations_labels_df.corr("pearson").iloc[9:, 0:9]

px.imshow(
    correlations_labels,
    labels=dict(x="", y="", color="Correlation"),
    x=correlations_labels.columns,
    y=correlations_labels.index,
    zmin=-1,
    zmax=1,
    color_continuous_scale="PRgn",
)

### Detalle Interesante 3: ¿Usamos Artista?

In [ ]:
df["artist"].value_counts()

In [ ]:
px.histogram(
    df,
    "artist",
)

In [ ]:
artista_ohe = OneHotEncoder(sparse=False)
artista_encoded = artista_ohe.fit_transform(df[["artist"]])
artista_cols = artista_ohe.get_feature_names_out()


artista_df = pd.DataFrame(artista_encoded, columns=artista_cols)
artista_df

In [ ]:
artista_df.info()

**Esta transformación produce 519 dimensiones. Una locura...**

> **Pregunta:** ¿La cantidad de dimensiones influirá en la calidad de clasificación que logremos?

Imagínense ahora las correlaciones de cada una de estas viariables con respecto a la variable de salida.

### Maldición de la Dimensionalidad

La maldición de la dimensionalidad es el problema que consiste en que a medida que aumentan las dimensiones, los datos tienden a hacerse cada vez más *sparse*/escasos sobre las dimensiones en las cuales están representados. Una simple analogía para entender esto es que:

> *A medida que aumenta la cantidad de features, aumenta el volumen en donde se encuentran los datos, haciendo que estos se separen bastante entre ellos. *



<div align='center'>
    <img src='./resources/curse.png' width=600/>
</div>

<div align='center'>
    Fuente: <a href='https://www.researchgate.net/figure/The-effect-of-the-curse-of-dimensionality-when-projected-in-1-one-dimension-2-two_fig3_342638066'> A comprehensive survey of anomaly detection techniques for high dimensional big data en Research Gate.</a>
</div>


Esto implica que, para poder seguir distinguiendo correctamente los datos, se debe aumentar masivamente su cantidad a medida que se aumentan las dimensiones.

**¿En qué nos afecta esto?**

Induce comunmente a una reducción del rendimiento de los clasificadores/regresores.


Más en Wikipedia: [Curse of dimensionality](https://en.wikipedia.org/wiki/Curse_of_dimensionality).

## Mejorando la Clasificación


Entonces, hasta acá tenemos 3 problemas: 

1. Hay características que "aportan" más o menos la misma información.
2. Hay características que podrían no aportar información para poder clasificar correctamente.
3. Tenemos una gran cantidad de dimensiones, lo cual podría entorpecer la clasificación.


Por ende, sería ideal eliminar un par de dimensiones con el fin de mejorar la clasificación. 


Para comparar las mejoras, utilizaremos un **Baseline**, el cual no es más que es un modelo inicial al cual , a medida que vayamos generando mejores modelos, nos iremos comparando (para ver si mejoramos y cuanto). Durante toda esta clase, nuestro **baseline** (modelo que compararemos) será **Tree** entrenado con todas las features.


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.tree import DecisionTreeClassifier

preprocessing = ColumnTransformer(
    [
        (
            "Scale",
            MinMaxScaler(),
            [
                "duration_ms",
                "tempo",
                "loudness",
                "time_signature",
            ],
        ),
        (
            "One Hot Encoding",
            OneHotEncoder(sparse=False, handle_unknown="ignore"),
            [
                "key",
                "mode",
                "artist",
                "time_signature",
            ],
        ),
    ],
    remainder="passthrough",
)

# Creamos nuestro baseline pipeline
baseline_pipe = Pipeline(
    steps=[
        ("Preprocessing", preprocessing),
        ("Tree", DecisionTreeClassifier(random_state=42)),
    ]
)

Aplicamos **holdout** al dataset

In [ ]:
from sklearn.model_selection import train_test_split

labels = df.loc[:, "genre"]
features = df.drop(columns=["genre", "id", "name"])

X_train, X_test, y_train, y_test = train_test_split(
    features,
    labels,
    test_size=0.2,
    random_state=42,
    shuffle=True,
    stratify=labels,
)

Ahora, definimos el ciclo de Entrenamiento y Evaluación.
La métrica con la cuál evaluaremos el desempeño del clasificador será `f1_score`.

Para esto, definiremos la función `train_and_evaluate` que dado un pipeline y conjuntos de entrenamiento y prueba, entrena un clasificador y retorna su evaluación:



In [ ]:
from sklearn.metrics import f1_score


def train_and_evaluate(
    pipe, print_=True, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test
):

    # notar que los datasets son parámetros por defecto.

    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)

    if print_:
        print("Matriz de confusión: \n")
        print(confusion_matrix(y_test, y_pred, labels=pipe.classes_))
        print("\nReporte de Clasificación: \n")
        print(
            classification_report(y_test, y_pred, target_names=pipe.classes_),
        )

    return f1_score(y_test, y_pred, average="macro")


train_and_evaluate(baseline_pipe)

> **Pregunta:❓** ¿Cómo sé que mi modelo es mejor que uno que clasifica al azar?


### Modelos Dummy

El [DummyClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyClassifier.html#sklearn.dummy.DummyClassifier) es un clasificador que ignora todas las features de entrada y genera salidas aleatorias como respuesta a las predicciones.

Permite saber si los modelos que estamos implementando son mejores que clasificar al azar. Por lo general, es una de las primeros chequeos que hacemos ya que permite anticipadamente saber si estamos generando modelos que aprenden o no.

Para la regresión, existe [DummyRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyRegressor.html#sklearn.dummy.DummyRegressor).

In [ ]:
from sklearn.dummy import DummyClassifier

dummy_pipe = Pipeline(
    steps=[
        ("Preprocessing", preprocessing),
        ("Tree", DummyClassifier(strategy="stratified")),
    ]
)

train_and_evaluate(dummy_pipe)


-------------

## Selección de atributos


Entonces, recordando los 3 problemas que habíamos mencionado previamente: 

1. Hay características que "aportan" más o menos la misma información.
2. Hay características que podrían no aportar información para poder clasificar correctamente.
3. Tenemos una gran cantidad de dimensiones, lo cual podría entorpecer la clasificación.

La idea es aplicar métodos de selección de características para que, de forma automatizada, se encuentre un subconjunto de atributos que mejoren los resultados de los modelos de clasificación/regresión implementados.

### Scheme independent o Método de Filtro .


Compara las características con las etiquetas a través de test estadísticos simples e ignora la relación entre las características en si. Para ocupar este tipo de técnicas se requiere una métrica y una estrategia

#### Métricas

- Varias métricas para clasificación: 
    - Anova (`f_classif`). La idea es calcular un estadístico F-score, el cuál indica que tan fácil es para un atributo distinguir entre clases. Más información [aquí](https://datascience.stackexchange.com/questions/74465/how-to-understand-anova-f-for-feature-selection-in-python-sklearn-selectkbest-w).
    - Mutual information (`mutual_info_classif`) es un estadístico que mide la independencia entre dos variables aleatorias. 0 indica independencia entre variables. Valores más altos indican mayor depencia. En general da mejores resultados que `f_classif`, pero su implementación es más lenta.
    - Chi squared (`chi2`) realiza un test estadístico $\chi^2$ que, al igual que la función anterior, mide la dependencia entre distintas variables. Sirve solo con variables categóricas (en OneHot) o conteos (como Bag of Words).
    
- Para regresión, se usan otro tipo de métricas especializadas en ellas.
- Referencia: [Univariate Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection).

#### Estrategias

Definen como se seleccionarán las mejores características: 

- `SelectKBest` selecciona las features con los mejores valores. Hay que especificar el número de features que seleccionaremos.

- `SelectPercentile` selecciona el porcentaje con mejores valores. Hay que especificar con cuanto porcentaje quedarse.

#### Integrar la Selección de Características al Pipeline

La búsqueda de mejores características se realiza al momento de entrenar un pipeline. Luego, al momento de predecir, el selector de características simplemente descarta las características no utilizadas antes de pasar a la siguiente etapa. El siguiente ejemplo muestra lo anteriormente dicho:

In [ ]:
from sklearn.feature_selection import (
    SelectKBest,
    SelectPercentile,
    f_classif,
    mutual_info_classif,
)

# Creamos nuestro baseline pipeline
selection_pipeline = Pipeline(
    steps=[
        ("Preprocessing", preprocessing),
        # Conservamos el 70% mejor según la métrica seleccionada
        ("Selection", SelectPercentile(f_classif, percentile=70)),
        ("Tree", DecisionTreeClassifier(random_state=42)),
    ]
)

train_and_evaluate(selection_pipeline)

In [ ]:
# Creamos nuestro baseline pipeline
selection_pipeline = Pipeline(
    steps=[
        ("Preprocessing", preprocessing),
        ("Selection", SelectPercentile(f_classif, percentile=20)),
        ("Tree", DecisionTreeClassifier(random_state=42)),
    ]
)

train_and_evaluate(selection_pipeline)

In [ ]:
# Creamos nuestro baseline pipeline
selection_pipeline = Pipeline(
    steps=[
        ("Preprocessing", preprocessing),
        ("Selection", SelectPercentile(mutual_info_classif, percentile=20)),
        ("Tree", DecisionTreeClassifier(random_state=42)),
    ]
)

train_and_evaluate(selection_pipeline)

> **Pregunta: ❓** ¿Cómo elegir el mejor porcentaje de features por conservar? ¿Y la métrica?

In [ ]:
selection_pipeline

In [ ]:
selection_pipeline.steps[1][1]

In [ ]:
selection_pipeline.steps[1][1].percentile

In [ ]:
f1 = []
for i in range(10, 101, 10):
    selection_pipeline.steps[1][1].percentile = i
    f1.append([i, train_and_evaluate(selection_pipeline, print_=False)])
f1 = np.array(f1)

px.line(
    x=f1[:, 0],
    y=f1[:, 1],
    title="F1 según cantidad de Features Conservadas",
)

## GridSearch

Si bien el ciclo anterior nos permitió encontrar el mejor valor para el porcentaje de la selección de atributos, es bastante trabajo implementarlo, pensando más aún que comunmente se quieren optimizar varias partes del pipeline y no solo un paso en específico.

En el caso anterior, un ejemplo de esto podría ser variar el porcentaje como la métrica usada, teniendo una malla de búsqueda del estilo:

| `f_classif` | `mutual_info_classif` |
|---|---|
| 10 | 10 |
| 20 | 20 |
| 30 | 30 |
| 40 | 40 |
| 50 | 50 |
| 60 | 60 |
| 70 | 70 |
| 80 | 80 |
| 90 | 90 |
| 100 | 100 |


Por esto, la idea es tener un mecanismo para el cual podamos pasarle una lista de hiperparámetros, que este lo pruebe todos y que retorne el mejor modelo. 
Este mecanismo en `scikit-learn` es conocido como Búsqueda de grilla o `Grid-search`.





### Encontrar los Parámetros Disponibles para Modificar

En general, cualquier clase de scikit-learn implementa la función `get_params`, la cual muestra los parámetros disponibles para probar y modificar.


In [ ]:
DecisionTreeClassifier().get_params()

In [ ]:
SelectPercentile().get_params()

Para el caso de una `Pipeline`, muestra las `steps` de la pipeline más los parámetros de cada una de las steps. 
Noten que los parámetros de cada `step` siguen la notación: `{nombre_step}__{parámetro_step}`

Ejemplo: Número de porcentajes que escogeremos - `Selection__percentile`



In [ ]:
selection_pipeline = Pipeline(
    steps=[
        ("Preprocessing", preprocessing),
        ("Selection", SelectPercentile(f_classif, percentile=20)),
        ("Tree", DecisionTreeClassifier(random_state=42)),
    ]
)

selection_pipeline.get_params()

La idea es definir un grilla de hiperparámetros para que GridSearch los explore y elija el mejor.

In [ ]:
param_grid = [
    {"Selection__percentile": range(10, 101, 5)}
]
param_grid

Y luego invocar GridSearch con el Pipeline, la grilla de hiperparámetros, la métrica (pueden ver las métricas disponibles [aquí](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter)) y la cantidad de cores de su CPU que deseen usar:

In [ ]:
from sklearn.model_selection import GridSearchCV

gs = GridSearchCV(selection_pipeline, param_grid, n_jobs=-1, scoring="f1_macro")
gs.fit(X_train, y_train)

In [ ]:
gs.best_score_

In [ ]:
gs.best_params_

Los resultados de la exploración de grilla los pueden visualizar así: 

In [ ]:
gs.best_estimator_

#### Ejemplo de Uso para Predicción de GridSearch

In [ ]:
# sacamos el vector de los atributos desde las features
gs.predict(X_test)

> **Pregunta ❓**: ¿Y si ahora quiero usar otra métrica univariada para seleccionar atributos?

Por ejemplo, [`Mutual information`](https://en.wikipedia.org/wiki/Mutual_information)
Vamos nuevamente a buscar el nombre del atributo que queremos modificar y ejecutamos nuevamente grid-search.

In [ ]:
selection_pipeline.get_params()

In [ ]:
from sklearn.feature_selection import f_classif, mutual_info_classif

param_grid = [
    {
        "Selection__percentile": range(5, 101, 5),
        "Selection__score_func": [f_classif, mutual_info_classif],
    }
]

In [ ]:
gs = GridSearchCV(selection_pipeline, param_grid, n_jobs=-1, scoring="f1_macro")
train_and_evaluate(gs)

In [ ]:
gs.best_score_

In [ ]:
gs.best_params_

### `Grid Search` y `CV` (Cross Validated)

`Grid Search` recibe como parámetro `X_train` y `y_train` y (a diferencia de la función que definimos en el baseline) entrena un modelo usando cross validation sobre todos los parámetros. 

<center>
    <img src='./resources/kfold.png' width=400/>
</center>

Es decir, por cada fracción del Cross Validation entrena el modelo y luego promedia los scores para obtener el score final de la configuración de hiperparámetros que estaba probando.



Una vez terminada la búsqueda de los hiperparámetros que generan el mejor modelo sobre los k-folds, **por defecto**, selecciona el mejor modelo encontrado **y entrena con todos los datos utilizados.**

**ESTO PUEDE CAUSAR DATA LEAKAGES**. 

Por eso es importante usar `GridSearchCV` con `X_train` y `y_train` y no con todo el dataset (aunque dentro de este se ejecute un Cross Validation). De todas formas, ustedes pueden controlar este comportamiento a través del parámetro `refit`.

In [ ]:
pd.DataFrame(gs.cv_results_)

> **Pregunta ❓**: ¿Podemos entonces también cambiar el clasificador y probar varios tipos?
    

### Probando con más clasificadores 

Efectivamente, la notación permite generar distintas grillas de búsqueda para distintos clasificadores a través de la definición de distintos diccionarios dentro de la lista que se le provee a GridSearch:


```
[
    # grilla 1
    {
     'modelo' : [Modelo1()],
     'modelo__param1': ...
    },
    # grilla 2
    {
     'modelo' : [Modelo1()],
     'modelo__param1': ...
    }, 
    ...

]
```

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

pipe = Pipeline(
    [
        ("preprocessing", preprocessing),
        ("selection", SelectPercentile(f_classif)),
        ("model", KNeighborsClassifier()),
    ]
)

pipe.get_params()

In [ ]:
DecisionTreeClassifier().get_params()

In [ ]:
grid = [
    # grilla 1: tree
    {
        "selection__percentile": range(10, 101, 10),
        "model": [DecisionTreeClassifier()],
        "model__criterion": ["gini", "entropy"],
    },
    # grilla 2: knn
    {
        "selection__percentile": range(10, 101, 10),
        "model": [KNeighborsClassifier()],
        "model__n_neighbors": [2, 4, 5, 10],
    },
    # grilla 3: random forest
    {
        "model": [RandomForestClassifier()],
        "model__criterion": ["gini", "entropy"],
        "model__bootstrap": [True, False],
    },
]

In [ ]:
gs2 = GridSearchCV(pipe, grid, n_jobs=-1, scoring="f1_macro").fit(X_train, y_train)

In [ ]:
gs2.best_score_

In [ ]:
gs2.best_estimator_

-------------------------

## Reducción de la dimensionalidad

Técnicas que reducen el número de características de forma no supervisadas.

- Eliminan ruido.
- Pueden mejorar el rendimiento de los modelos, sobre todo si se tienen muchas dimensiones.
- Pueden proyectar datos en dos/tres dimensiones.

**Problemas** con algoritmos de reducción de dimensionalidad: 


- Las dimensiones ya no son intepretables.



### Principal Component Analysis


Reduce dimensiones de nuestro dataset tratando de no perder mucha información.

Objetivo. Encontrar direcciónes tales que al proyectar nuestros datos, la varianza de los puntos proyectados se maximize.

<center>
<img src='./resources/pca.jpg' width=600/>
    </center>
    
<center>
Fuente: <a href='https://devopedia.org/principal-component-analysis'>https://devopedia.org/principal-component-analysis</a>

</center>

In [ ]:
fig = px.scatter(df, x="energy", y="loudness", color="genre")

fig.show()

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA()
components = pca.fit_transform(df[["energy", "loudness"]])
labels_ = {
    ["x", "y"][i]: f"PC {i+1} ({var:.1f}%)"
    for i, var in enumerate(pca.explained_variance_ratio_ * 100)
}

fig = px.scatter(
    x=-components[:, 0], y=-components[:, 0], color=df["genre"], labels=labels_
)


fig.show()

### Varianza explicada

**¿Cuánta información mantenemos después de ejecutar PCA?**

Cada componente explica una cierta cantidad de varianza de los datos.
Esto esta determinado por los autovalores $\lambda$

Los componentes con mayor varianza incuirán mayor información.


In [ ]:
n_components = 3

pca = PCA(n_components=n_components)
components = pca.fit_transform(preprocessing.fit_transform(features))

total_var = pca.explained_variance_ratio_.sum() * 100

labels_ = [
    f"PC {i+1} = {var:.1f}%"
    for i, var in enumerate(pca.explained_variance_ratio_ * 100)
]

fig = px.scatter_matrix(
    components,
    color=df["genre"],
    dimensions=range(n_components),
    labels=labels_,
    title=f"Varianza Explicada Total: {total_var:.2f}%.<br>Varianza por Componente: {labels_}",
    height=800,
)
fig.update_traces(diagonal_visible=False)

fig.show()

### ¿Cómo encontrar la mejor cantidad de dimensiones?

Vamos viendo cuanta varianza acumula cada componente.

En algún punto, la varianza marginal que acumula el siguiente es mucho mas baja que la anterior. 
Ese es el punto indicado en donde cortar.


Esto lo podemos ver en el siguiente gráfico:


In [ ]:
pca = PCA()
pca.fit(preprocessing.fit_transform(features))
exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)

px.area(
    x=range(1, exp_var_cumul.shape[0] + 1),
    y=exp_var_cumul,
    labels={"x": "Número de componentes", "y": "Varianza explicada"},
)

Mas o menos en los 8 componentes principales ya se explican mas del 90% de la varianza del dataset.

In [ ]:
# Creamos nuestro baseline pipeline

pca_pipeline = Pipeline(
    steps=[
        ("Preprocessing", preprocessing),
        ("Reduccion", PCA()),
        ("Tree", DecisionTreeClassifier(random_state=42)),
    ]
)

In [ ]:
pca_pipeline.get_params()

In [ ]:
param_grid = [
    {"Reduccion__n_components": [10, 20, 30, 40, 50, 100, 200, 300, 400]}
]
gs_pca = GridSearchCV(pca_pipeline, param_grid=param_grid, scoring='f1_weighted', n_jobs=-1)

train_and_evaluate(gs_pca)

In [ ]:
gs_pca.best_estimator_

### Mejoras a GridSearch

#### `HalvingGridSearchCV`


![](./resources/halvinggscv.png)

In [ ]:
from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingGridSearchCV

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.pipeline import Pipeline

selection_pipeline = Pipeline(
    steps=[
        ("preprocessing", preprocessing),
        ("selection", SelectPercentile(f_classif, percentile=20)),
        ("model", RandomForestClassifier()),
    ]
)

param_grid = [
    {
        "selection__percentile": range(5, 101, 10),
        "selection__score_func": [f_classif, mutual_info_classif],
        "model__criterion": ["gini", "entropy"],
        "model__bootstrap": [True, False],
    }
]

hgs = HalvingGridSearchCV(selection_pipeline, param_grid, n_jobs=-1, scoring='f1_macro')

train_and_evaluate(hgs)

### Proyectos Relacionados

https://scikit-learn.org/stable/related_projects.html

- `scikit-optimize`: A library to minimize (very) expensive and noisy black-box functions. It implements several methods for sequential model-based optimization, and includes a replacement for GridSearchCV or RandomizedSearchCV to do cross-validated parameter search using any of these strategies.

- `sklearn-deap` Use evolutionary algorithms instead of gridsearch in scikit-learn.


- [`optuna`](https://optuna.readthedocs.io/en/stable/index.html): Optuna is an automatic hyperparameter optimization software framework, particularly designed for machine learning. It features an imperative, define-by-run style user API. Thanks to our define-by-run API, the code written with Optuna enjoys high modularity, and the user of Optuna can dynamically construct the search spaces for the hyperparameters.